In [4]:
import yaml
import requests
import pandas as pd

In [5]:
API_KEY = yaml.full_load(open('creds.yaml', 'r'))['maps_api']

In [6]:
dd = requests.get('https://gbfs.baywheels.com/gbfs/en/station_information.json')
stations = pd.DataFrame(dd.json()['data']['stations'])

In [7]:
stations.loc[stations['name'].str.contains('Caltrain')]

,lon,legacy_id,region_id,electric_bike_surcharge_waiver,rental_uris,eightd_station_services,station_type,eightd_has_key_dispenser,station_id,rental_methods,short_name,capacity,has_kiosk,external_id,lat,name
88,-122.394495,465,3,False,"{'ios': 'https://sfo.lft.to/lastmile_qr_scan',...",[],classic,False,a7da1857-747a-4507-9d8f-2a9f95f0babf,"[KEY, CREDITCARD]",SF-J29-3,18,True,a7da1857-747a-4507-9d8f-2a9f95f0babf,37.776311,San Francisco Caltrain Station (King St at 4th...
186,-122.391885,130,3,False,"{'ios': 'https://sfo.lft.to/lastmile_qr_scan',...",[],classic,False,e7691d93-eed6-4388-b681-00ae0b455d23,"[KEY, CREDITCARD]",SF-P30,35,True,e7691d93-eed6-4388-b681-00ae0b455d23,37.757704,22nd St Caltrain Station
360,-122.395282,30,3,False,"{'ios': 'https://sfo.lft.to/lastmile_qr_scan',...",[],classic,False,ea11aa7f-c070-46a4-a074-70604340efa2,"[KEY, CREDITCARD]",SF-J29,19,True,ea11aa7f-c070-46a4-a074-70604340efa2,37.776598,San Francisco Caltrain (Townsend St at 4th St)


In [8]:
lat1, lon1 = stations.loc[stations['name'] == 'Commercial St at Montgomery St'][['lat', 'lon']].iloc[0]

In [9]:
lat2, lon2 = stations.loc[stations['name'] == 'San Francisco Caltrain Station (King St at 4th St)'][['lat', 'lon']].iloc[0]

In [10]:
req = requests.get("https://maps.googleapis.com/maps/api/directions/json?avoid=highways&destination={},{}&mode=bicycling&origin={},{}&key={}".format(lat1, lon1, lat2, lon2, API_KEY))

In [11]:
res = req.json()

In [12]:
res

{'geocoded_waypoints': [{'geocoder_status': 'OK',
   'place_id': 'ChIJiZsGldZ_j4ARSAUO2zvLtxM',
   'types': ['establishment', 'point_of_interest']},
  {'geocoder_status': 'OK',
   'place_id': 'ChIJAyk-84qAhYARoYww6NIvE6A',
   'types': ['establishment', 'point_of_interest']}],
 'routes': [{'bounds': {'northeast': {'lat': 37.7968231, 'lng': -122.3876902},
    'southwest': {'lat': 37.7762883, 'lng': -122.4028589}},
   'copyrights': 'Map data ©2022 Google',
   'legs': [{'distance': {'text': '2.4 mi', 'value': 3820},
     'duration': {'text': '15 mins', 'value': 917},
     'end_address': 'Commercial St at Montgomery St, San Francisco, CA 94111, USA',
     'end_location': {'lat': 37.7942628, 'lng': -122.4028589},
     'start_address': 'San Francisco Caltrain Station (King St at 4th St), San Francisco, CA 94107, USA',
     'start_location': {'lat': 37.7762883, 'lng': -122.394524},
     'steps': [{'distance': {'text': '308 ft', 'value': 94},
       'duration': {'text': '1 min', 'value': 17},
 

In [13]:
res.keys()

dict_keys(['geocoded_waypoints', 'routes', 'status'])

In [14]:
import datetime as dt

str(dt.timedelta(seconds=res['routes'][0]['legs'][0]['duration']['value']))

'0:15:17'

In [33]:
def convert_sec_to_time(num_sec):
    return str(dt.timedelta(seconds=res['routes'][0]['legs'][0]['duration']['value']))


def get_travel_time(req):
    return req['routes'][0]['legs'][0]['duration']['value']


def get_stations_coord(name, stations_df):
    if name in stations_df['name'].values:
        return stations_df.loc[stations_df['name'] == name][['lat', 'lon']].iloc[0]
    return None

In [34]:
lat, lon = get_stations_coord('Commercial St at Montgomery St', stations)

In [35]:
lat

37.7942465

In [36]:
lon

-122.402855